In [1]:
#First let's import the necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
from IPython.display import display, HTML

pd.set_option('display.max_columns', 500)

In [2]:
#Specifying the Data Path

cwd = os.getcwd()
file_path = os.path.join(cwd, 'cleaned_speed_dating.csv')

In [3]:
df=pd.read_csv(file_path)

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, random_state=42)

In [7]:
female_df = df.loc[df['gender'] == 0]
male_df = df.loc[df['gender'] == 1]

In [8]:
female_train, female_test = train_test_split(female_df, test_size = 0.2, random_state=42)
male_train, male_test = train_test_split(male_df, test_size = 0.2, random_state=42)

# Logistic Regression

In [9]:
from sklearn import linear_model
lr = linear_model.LogisticRegression()

In [10]:
predictors = ['attr_partner']
lr_model = lr.fit(train[predictors].values, train['dec'].values)
print('training set performance is {}'.format(lr_model.score(train[predictors].values, train['dec'].values)))
print('test set performance is {}'.format(lr_model.score(test[predictors].values, test['dec'].values)))

training set performance is 0.7276960327807785
test set performance is 0.7297096053611318


In fact, if you only use one variable, that is no different than just doing a cut off based on one of the previous graphs and make naive predictions based on that.

In [11]:
list(df.columns[1:-1])

['gender',
 'age',
 'date',
 'sports',
 'tvsports',
 'exercise',
 'dining',
 'museums',
 'art',
 'hiking',
 'gaming',
 'clubbing',
 'reading',
 'tv',
 'theater',
 'movies',
 'concerts',
 'music',
 'shopping',
 'yoga',
 'attr_want',
 'sinc_want',
 'intel_want',
 'fun_want',
 'amb_want',
 'shar_want',
 'attr_self',
 'sinc_self',
 'fun_self',
 'intel_self',
 'amb_self',
 'pid',
 'age_partner',
 'int_corr',
 'samerace',
 'attr_partner',
 'sinc_partner',
 'intel_partner',
 'fun_partner',
 'amb_partner',
 'shar_partner',
 'prob']

In [13]:
predictors = ['age','date','int_corr',
 'samerace',
 'sinc_partner',
 'intel_partner',
 'fun_partner',
 'amb_partner',
 'shar_partner','prob']
lr_model = lr.fit(train[predictors].values, train['dec'].values)
print('training set performance is {}'.format(lr_model.score(train[predictors].values, train['dec'].values)))
print('test set performance is {}'.format(lr_model.score(test[predictors].values, test['dec'].values)))

training set performance is 0.7135406965915441
test set performance is 0.7073715562174236


In [14]:
# You can also see how important each of those factors is (sort of)
print(lr_model.coef_)

[[-0.01254919 -0.01858074  0.03288736  0.06893665 -0.07068834  0.04977025
   0.42423327 -0.15730692  0.258164    0.15273089]]


But it does seems like attractiveness is more indicative than anything else. Next you can try to combine them and repeat the same procedure. Is there an improvement to the performance? What can you infer from the coefficients this time? 

Try to include also those variables that you think are important and repeat the same step again. Observe what happen to the performances when you add more and more predictors.

Based on the graphs on EDA, it seems that male and female make their decisions quite differently. Try to repeat the above with female_df and male_df and see if the results improve.

# Benchmark

In order to know how good our prediction performance is, we should at least compare it to the performances of some naive algorithms. 

In [88]:
# what if I just look at the training set and guess the most popular decisions?
no_female_train = female_train.query('dec == 0')
print('Proportion of rejection by female in training set is {}'\
      .format(yes_female_train.shape[0]/female_train.shape[0]))

no_female_test = female_test.query('dec == 0')
print('Proportion of rejection by female in test set is {}'\
      .format(yes_female_test.shape[0]/female_test.shape[0]))

Proportion of rejection by female in training set is 0.6255690440060698
Proportion of rejection by female in test set is 0.6236722306525038


In [89]:
# what if I just look at the training set and guess the most popular decisions?
no_male_train = male_train.query('dec == 0')
print('Proportion of rejection by male in training set is {}'\
      .format(yes_male_train.shape[0]/male_train.shape[0]))

no_male_test = male_test.query('dec == 0')
print('Proportion of rejection by male in test set is {}'\
      .format(yes_male_test.shape[0]/male_test.shape[0]))

Proportion of rejection by male in training set is 0.5162824734723747
Proportion of rejection by male in test set is 0.49122807017543857


In [106]:
# what if I simply do a cut off at attr_partner and base my decision on that? (refer to graphs plotted in EDA)
male_test['attr_cut_predict'] = (male_test['attr_partner']>=7)
print('male test set performance is {}'\
      .format((male_test['attr_cut_predict'] == male_test['dec']).sum()/male_test.shape[0]))

test set performance is 0.7532846715328467


/home/weizhen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [111]:
female_test['attr_cut_predict'] = (female_test['attr_partner']>=8)
print('female test set performance is {}'\
      .format((female_test['attr_cut_predict'] == female_test['dec']).sum()/female_test.shape[0]))

female test set performance is 0.7329286798179059


/home/weizhen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Tree

Tree model allows combination of factors (as opposed to logistic regression model).

In [54]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(min_impurity_split=0.3)

In [55]:
predictors = ['age','date','int_corr',
 'samerace',
 'sinc_partner',
 'intel_partner',
 'fun_partner',
 'amb_partner',
 'shar_partner','prob', 'attr_partner','attr_want',
 'sinc_want',
 'intel_want',
 'fun_want',
 'amb_want',
 'shar_want']
dt_model = dt.fit(train[predictors].values, train['dec'].values)
print('training set performance is {}'.format(dt_model.score(train[predictors].values, train['dec'].values)))
print('test set performance is {}'.format(dt_model.score(test[predictors].values, test['dec'].values)))

training set performance is 0.8794933879679643
test set performance is 0.7267311988086373


In [56]:
dt_model.feature_importances_

array([ 0.02697355,  0.01483152,  0.04568818,  0.00881074,  0.01654893,
        0.01588581,  0.03525998,  0.02560737,  0.11024847,  0.05146821,
        0.42485679,  0.05590846,  0.03568541,  0.04481684,  0.03602042,
        0.02073288,  0.03065645])

# Ensemble Methods

In [58]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [59]:
predictors = ['age','date','int_corr',
 'samerace',
 'sinc_partner',
 'intel_partner',
 'fun_partner',
 'amb_partner',
 'shar_partner','prob', 'attr_partner','attr_want',
 'sinc_want',
 'intel_want',
 'fun_want',
 'amb_want',
 'shar_want']
rf_model = rf.fit(train[predictors].values, train['dec'].values)
print('training set performance is {}'.format(rf_model.score(train[predictors].values, train['dec'].values)))
print('test set performance is {}'.format(rf_model.score(test[predictors].values, test['dec'].values)))

training set performance is 0.987707208046191
test set performance is 0.7580044676098288


In [73]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(max_depth=7)

In [74]:
predictors = ['age','date','int_corr',
 'samerace',
 'sinc_partner',
 'intel_partner',
 'fun_partner',
 'amb_partner',
 'shar_partner','prob', 'attr_partner','attr_want',
 'sinc_want',
 'intel_want',
 'fun_want',
 'amb_want',
 'shar_want']
gb_model = gb.fit(train[predictors].values, train['dec'].values)
print('training set performance is {}'.format(gb_model.score(train[predictors].values, train['dec'].values)))
print('test set performance is {}'.format(gb_model.score(test[predictors].values, test['dec'].values)))

training set performance is 0.979698267833861
test set performance is 0.7944899478778853
